In [13]:
%matplotlib inline  
from fastai.vision import *
from fastai.data_block import _maybe_squeeze
from fastai.callbacks import *
from sklearn.model_selection import GroupKFold
from joblib import load, dump
from efficientnet_pytorch import EfficientNet
from fastai.tabular import * 
def strt_split(x, y, n_folds=10, random_seed = 42, path=Path('')):  
    try: 
        val_name = load('val_idx_10.joblib')
    except:
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
        val_name = [(val_idx, trn_idx) for trn_idx, val_idx in skf.split(x, y)]
        dump(val_name,'val_idx_10.joblib')
    return val_name

def modified_label_from_df(self, cols:IntsOrStrs=1, label_cls:Callable=None, **kwargs):
    "Label `self.items` from the values in `cols` in `self.inner_df`."
    self.inner_df.labels.fillna('', inplace=True)
    labels = self.inner_df.iloc[:,df_names_to_idx(cols, self.inner_df)]
    assert labels.isna().sum().sum() == 0, f"You have NaN values in column(s) {cols} of your dataframe, please fix it."
    if is_listy(cols) and len(cols) > 1 and (label_cls is None or label_cls == MultiCategoryList):
        new_kwargs,label_cls = dict(one_hot=True, classes= cols),MultiCategoryList
        kwargs = {**new_kwargs, **kwargs}
    return self._label_from_list(_maybe_squeeze(labels), label_cls=label_cls, **kwargs)

In [2]:
df = pd.read_feather('df_trn.fth')

In [3]:
col_drop = ['SOPInstanceUID', 
            'StudyID', 
            'SamplesPerPixel', 
            'PhotometricInterpretation',
            'BitsAllocated', 
            'MultiImageOrientationPatient', 
            'MultiPixelSpacing', 
            'MultiWindowCenter', 
            'MultiWindowWidth', 
            'Modality', 
            'MultiImagePositionPatient',
            'StudyInstanceUID', 
            'SeriesInstanceUID', 
            'RescaleSlope', 
            'PatientID'] 

In [4]:
df.drop(col_drop, axis=1, inplace = True)

In [5]:
df.columns

Index(['ImagePositionPatient', 'ImageOrientationPatient', 'Rows', 'Columns',
       'PixelSpacing', 'BitsStored', 'HighBit', 'PixelRepresentation',
       'WindowCenter', 'WindowWidth', 'RescaleIntercept', 'fname',
       'ImagePositionPatient1', 'ImagePositionPatient2',
       'ImageOrientationPatient1', 'ImageOrientationPatient2',
       'ImageOrientationPatient3', 'ImageOrientationPatient4',
       'ImageOrientationPatient5', 'PixelSpacing1', 'img_min', 'img_max',
       'img_mean', 'img_std', 'WindowCenter1', 'WindowWidth1'],
      dtype='object')

In [6]:
df.nunique()

ImagePositionPatient          1307
ImageOrientationPatient         17
Rows                             9
Columns                          8
PixelSpacing                   222
BitsStored                       2
HighBit                          2
PixelRepresentation              2
WindowCenter                    37
WindowWidth                     44
RescaleIntercept                 4
fname                       674258
ImagePositionPatient1         8329
ImagePositionPatient2       368135
ImageOrientationPatient1        20
ImageOrientationPatient2        24
ImageOrientationPatient3        23
ImageOrientationPatient4       117
ImageOrientationPatient5       164
PixelSpacing1                  222
img_min                        257
img_max                       8298
img_mean                    669252
img_std                     670354
WindowCenter1                   34
WindowWidth1                    36
dtype: int64

In [7]:
cont = ['ImagePositionPatient2', 'ImagePositionPatient1', 'img_min', 'img_max', 'img_mean', 'img_std']

In [8]:
df['fname'] = df['fname'].apply(lambda x: f"{x.split('/')[-1][:-4]}.png")
PATH = Path('..')
DF_TRAIN =      pd.read_csv(PATH/'train_labels_as_strings.csv')
DF_TRAIN.columns = ['fname', 'labels']                            

In [9]:
df = pd.merge(df, DF_TRAIN, on='fname')

In [11]:
FOLD=0
VAL_IDX = strt_split(df['fname'], DF_TRAIN['labels'])[FOLD]
df.drop('fname', axis=1, inplace=True)

In [15]:
TabularList.label_from_df = modified_label_from_df

In [ ]:
Tab

In [ ]:
PATH = Path('..')
BS =            184
SZ =            380
FOLD =0
EXP_NAME =      f'NB_EXP_310_CV_{FOLD}_{SZ}'
IMG_TRAIN_224 = PATH/f'train_images_3chn_bg_{SZ}'
IMG_TEST_224  = PATH/f'test_images_3chn_bg_{SZ}'
DF_TRAIN =      pd.read_csv(PATH/'train_labels_as_strings.csv')
DF_SUBMI =      pd.read_csv(PATH/'stage_1_sample_submission.csv')



DF_SUBMI['fn'] = DF_SUBMI.ID.apply(lambda x: '_'.join(x.split('_')[:2]) + '.png')
DF_TRAIN['labels'].fillna('', inplace=True)


In [ ]:
ItemList.label_from_df = modified_label_from_df
test_fns = DF_SUBMI.fn.unique()
tfms = get_transforms(max_rotate=180, flip_vert=True, max_zoom=1.3)

data = (ImageList.from_csv('..', 'train_labels_as_strings.csv', folder=IMG_TRAIN_224.name)
        .split_by_idxs(valid_idx=VAL_IDX[0], train_idx=VAL_IDX[1])
        .label_from_df(label_delim=' ')
        .transform(tfms = tfms,size=SZ)
        .add_test('../' +IMG_TEST_224.name + '/' + test_fns)
        .databunch(bs=BS)
        .normalize(imagenet_stats))

        

In [ ]:
md_ef =  EfficientNet.from_pretrained('efficientnet-b4', num_classes=data.c)
#opt_func = partial(Ranger, betas=(0.92,0.99), eps=1e-6)

In [ ]:
data.show_batch()

In [ ]:
learn = Learner(data,
                md_ef,
                wd=1e-2,
                bn_wd=False, 
                true_wd=True, 
                metrics=[accuracy_thresh])

weights = [2, 1, 1, 1, 1, 1]
learn.loss_func  = torch.nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor(weights).cuda())
learn.model = nn.DataParallel(learn.model)
learn.to_fp16()
learn.unfreeze()


In [ ]:
lr = 1e-2/3
learn.recorder.plot()
plt.axvline(lr)

In [ ]:
learn.unfreeze()
SUFFIX = 'UNFRZ'
learn.fit_one_cycle(7,
                    lr, 
                    callbacks=[SaveModelCallback(learn, every='improvement', monitor='valid_loss', name = f'{EXP_NAME}_{SUFFIX}')])

In [ ]:
def get_preds(learn:Learner, sub_fn: str=f'{EXP_NAME}_PHASE_1_COS', TTA: bool = False, dt_type = DatasetType.Test):
    if TTA:
        learn.to_fp32()
        preds, targs = learn.TTA(ds_type=dt_type)
        sub_fn = f'{sub_fn}_TTA'
    else:
        preds, targs = learn.get_preds(dt_type)
    ids = []
    labels = []

    for fn, pred in zip(test_fns, preds):
        for i, label in enumerate(data.train_ds.classes):
            ids.append(f"{fn.split('.')[0]}_{label}")
            predicted_probability = '{0:1.10f}'.format(pred[i].item())
            labels.append(predicted_probability)
    pd.DataFrame({'ID': ids, 'Label': labels}).to_csv(f'{sub_fn}.csv', index=False)

In [ ]:
get_preds(learn)

In [ ]:
get_preds(learn, TTA=True)

In [ ]:
learn.load('NB_EXP_310_CV_0_380_UNFRZ')
learn.unfreeze()
learn.lr_find()

In [ ]:
lr = 1e-4/5
learn.recorder.plot()
plt.axvline(lr)

In [ ]:
learn.unfreeze()
SUFFIX = 'UNFRZ'
learn.fit_one_cycle(5,
                    lr, 
                    callbacks=[SaveModelCallback(learn, every='improvement', monitor='valid_loss', name = f'{EXP_NAME}_{SUFFIX}')])

In [ ]:
def get_preds(learn:Learner, sub_fn: str=f'{EXP_NAME}_PHASE_2_COS', TTA: bool = False, dt_type = DatasetType.Test):
    if TTA:
        learn.to_fp32()
        preds, targs = learn.TTA(ds_type=dt_type)
        sub_fn = f'{sub_fn}_TTA'
    else:
        preds, targs = learn.get_preds(dt_type)
    ids = []
    labels = []

    for fn, pred in zip(test_fns, preds):
        for i, label in enumerate(data.train_ds.classes):
            ids.append(f"{fn.split('.')[0]}_{label}")
            predicted_probability = '{0:1.10f}'.format(pred[i].item())
            labels.append(predicted_probability)
    pd.DataFrame({'ID': ids, 'Label': labels}).to_csv(f'{sub_fn}.csv', index=False)

In [ ]:
get_preds(learn)

In [ ]:
get_preds(learn, TTA=True)

In [ ]:
!sudo shutdown